In [ ]:
import whisper
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
from jiwer import wer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from jiwer import mer, wil
import os
import nemo.collections.asr as nemo_asr